In [1]:
%load_ext autoreload
%autoreload 2

from deribit_vol_surface import get_vol_surface
import numpy as np
from pyquant.black_scholes import *
from pyquant.common import *
from pyquant.sabr import Backbone, SABRCalc
from pyquant.ssvi import SSVICalc, SSVIParams, Eta, Lambda, Alpha, Beta, Gamma_
from pyquant.svi import SVICalc
from pyquant.vol_surface import *

from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True) 
import plotly.io as pio
pio.renderers.default = 'iframe'

In [2]:
vol_surface_chain_space = get_vol_surface("deribit_vol_surface.csv")
# convert to delta-space
vol_surface_delta_space: VolSurfaceDeltaSpace = SABRCalc().surface_to_delta_space(
    vol_surface_chain_space, Backbone(1.0)
)

# get all market with usable format
ssvi = SSVICalc()
calc_params, calibration_error, strikes, Ts, implied_variances, thetas = ssvi.calibrate(vol_surface_delta_space, 10, True)
ivs = np.sqrt(implied_variances/Ts)

ssvi_params = SSVIParams(
    Eta(calc_params[0]),
    Lambda(calc_params[1]), 
    Alpha(calc_params[2]), 
    Beta(calc_params[3]), 
    Gamma_(calc_params[4])
)

# считаем по тем же параметрам рынка сетку уже моделью
Z = ssvi._grid_implied_variances(
    ssvi_params, 
    StrikesMaturitiesGrid(Spot(vol_surface_delta_space.S), TimesToMaturity(Ts), Strikes(strikes)), thetas)

Z = np.sqrt(Z/Ts)

ivs, strikes, Ts, thetas, implied_variances, Z  = ivs[5:], strikes[5:], Ts[5:], thetas[5:], implied_variances[5:], Z[5:]



Implied variances to calibrate to: [6.17716273e-04 4.20943471e-04 3.60679404e-04 4.23746882e-04
 6.26080351e-04 4.41702202e-02 3.16116067e-02 2.72926749e-02
 2.95834138e-02 3.85795674e-02 8.59382970e-02 6.28264898e-02
 5.45657837e-02 5.70625407e-02 7.06250790e-02 1.27890273e-01
 9.74437242e-02 8.64669946e-02 8.91139751e-02 1.06639036e-01
 1.75711784e-01 1.33604170e-01 1.19084083e-01 1.23548213e-01
 1.49841534e-01 2.21639899e-01 1.68659314e-01 1.51075140e-01
 1.57648358e-01 1.93170358e-01 2.65758877e-01 2.03207770e-01
 1.83191987e-01 1.92396409e-01 2.37753992e-01 3.10620213e-01
 2.38808471e-01 2.16775784e-01 2.29879910e-01 2.87417505e-01
 3.57139048e-01 2.75918638e-01 2.52122397e-01 2.70506788e-01
 3.43007065e-01 4.04467053e-01 3.13784788e-01 2.88382908e-01
 3.12809348e-01 4.01907680e-01]
Strikes from delta-space we calibrate to: [1760.30244267 1792.12196553 1816.71212101 1842.50241728 1876.49909563
 1420.95760002 1639.62882714 1819.54411995 2073.8125857  2385.94226778
 1306.70093152 15

In [3]:
Z

array([0.64786543, 0.5622456 , 0.51833415, 0.5214725 , 0.57426499,
       0.64305818, 0.55943565, 0.51852585, 0.52402293, 0.57563963,
       0.65331594, 0.57167458, 0.53296052, 0.54116194, 0.59407095,
       0.66030358, 0.57826666, 0.54161637, 0.552777  , 0.6100308 ,
       0.66159395, 0.58013539, 0.54559148, 0.55941509, 0.61999923,
       0.6612865 , 0.58090155, 0.5483826 , 0.56481882, 0.62836089,
       0.66213587, 0.58278612, 0.552191  , 0.57152593, 0.63858831,
       0.66409718, 0.58567778, 0.55693367, 0.57947313, 0.65057227,
       0.66590522, 0.58841994, 0.56145684, 0.58726748, 0.66231896])

In [4]:
ivs

array([0.65990899, 0.55826776, 0.51873113, 0.54006174, 0.61673407,
       0.65145884, 0.55701258, 0.51910332, 0.53084675, 0.59057231,
       0.64907767, 0.56657166, 0.53370724, 0.54181475, 0.59270149,
       0.65898266, 0.57462372, 0.54250082, 0.55257569, 0.60854072,
       0.66203586, 0.57751425, 0.54658039, 0.55834453, 0.61805594,
       0.66181532, 0.57871279, 0.54947276, 0.56310763, 0.62597478,
       0.6624498 , 0.58084864, 0.55340553, 0.56988683, 0.63722781,
       0.66446903, 0.58404538, 0.55829249, 0.5782893 , 0.65118983,
       0.66670272, 0.58722779, 0.56295725, 0.58631434, 0.66459001])

In [6]:
# Создаем сетку для Surface
import plotly.graph_objects as go

from scipy.interpolate import griddata


xi = np.linspace(strikes.min(), strikes.max(), 50)  # 100 точек вместо 25
yi = np.linspace(Ts.min(), Ts.max(), 50)

xi, yi = np.meshgrid(xi, yi)

# Интерполируем ivs на сетку
zi = griddata((strikes, Ts), Z, (xi, yi), method='linear')


fig = go.Figure(data=[
        go.Scatter3d(z=ivs, x=strikes, y=Ts, mode='markers'),
        go.Surface(z=zi, x=xi, y=yi, opacity=0.8)


])

fig.update_layout(title='SSVI Volatility Surface', template="plotly_white", width=1000, height=1000)
fig.update_scenes(xaxis_title_text='Strike, USD',
                  yaxis_title_text='Time to maturity, years',
                  zaxis_title_text='Implied volatility',
                 zaxis=dict(range=[0.4, 0.8])
                 )

                    

fig.show()